<a href="https://colab.research.google.com/github/jeniferGoncalvesDaSilvaDev/python-finance/blob/main/aula_2_quantas_lojas_a_renner_consegue_abrir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Renner: Como projetar a receita da empresa através da abertura de novas lojas.
  
  
### Passo a passo:

   **Passo 1** - Ler e tratar os dados de lojas atuais.

   **Passo 2** - Ler e tratar dados dos munícipios brasileiros.
   
   **Passo 3** - Observar as características atuais das lojas.
   
   **Passo 4** - Concluir padrões observados.
   
   **Passo 5** - Traçar um plano de análise.
   
   **Passo 6** -  Pegar os dados faltantes para a projeção.
   
   **Passo 7** - Fazer a projeção.
   
   **Passo 8** - Usar a projeção pra projetar a receita líquida e chegar ao valor justo da empresa.

In [ ]:
!pip install mplcyberpunk

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use('cyberpunk')

# Coleta e tratamento de dados

----------

<br>

## Passo 1: Pegar os dados de lojas atuais.

In [ ]:
nome_sheet = 'Lista de Lojas | Stores List'

planilha_renner = pd.read_excel("lojas_renner.xlsx", sheet_name = nome_sheet, skiprows = 5,
                                usecols = ['Business', 'Country', 'UF', 'State', 'City',
                                           'Opening date', 'Shopping Mall / Street'])

In [ ]:
planilha_renner = planilha_renner.dropna()

planilha_renner = planilha_renner[(planilha_renner['Business'] == "Renner") &
                                  (planilha_renner['Country'] == "Brazil")]

planilha_renner['Opening date'] = pd.to_datetime(planilha_renner['Opening date'])

planilha_renner

## Passo 2: Pegar os dados dos municípios brasileiros.

Vamos no SIDRA, do IBGE.

https://sidra.ibge.gov.br/pesquisa/censo-demografico/demografico-2022/primeiros-resultados-populacao-e-domicilios

<br>

https://sidra.ibge.gov.br/pesquisa/pib-munic/tabelas

In [ ]:
pib_municipios = pd.read_excel("pib_municipios.xlsx")
pop_municipios = pd.read_excel("populacao_municipios.xlsx")

In [ ]:
dados_municipios = pib_municipios.merge(pop_municipios, on = 'City')

In [ ]:
dados_municipios['pib_per_capta'] = dados_municipios['pib']/dados_municipios['pop']

In [ ]:
dados_municipios[['City', 'UF']] = dados_municipios['City'].str.split("(", expand = True)

In [ ]:
dados_municipios['UF'] = dados_municipios['UF'].str.replace(")", "")

In [ ]:
planilha_renner['City'] = planilha_renner['City'].str.strip()
planilha_renner['UF'] = planilha_renner['UF'].str.strip()
dados_municipios['City'] = dados_municipios['City'].str.strip()
dados_municipios['UF'] = dados_municipios['UF'].str.strip()

### Juntando os dados

In [ ]:
dados_lojas_e_economicos = dados_municipios.merge(planilha_renner, on = ['City', 'UF'])

dados_lojas_e_economicos

# Análise exploratória dos dados

------------------

## Passo 3: Observar as características atuais das lojas.

### % em shoppings

In [ ]:
quantidade_de_lojas = len(dados_lojas_e_economicos)

In [ ]:
qtd_por_tipo = dados_lojas_e_economicos.groupby('Shopping Mall / Street')['Shopping Mall / Street'].size()

In [ ]:
qtd_por_tipo/quantidade_de_lojas

## A Renner abre lojas em cidades com qual perfil de renda e população?

### Menor e maior cidade com uma Renner

In [ ]:
dados_lojas_e_economicos.sort_values('pop')

### Cidade mais rica e mais pobre com uma Renner

In [ ]:
dados_lojas_e_economicos.sort_values('pib_per_capta')

## Em relação as cidades brasileiras, como é a distribuição de lojas abertas pela Renner em relação ao pib per capta? Ela foge de cidades "pobres"?
<br>
Pra visualizar esses dados, vamos igualar o perfil da amostra e retirar outliers.

In [ ]:
dados_municipios_histograma = dados_municipios[dados_municipios['pib_per_capta'] < 100]
#cidades muito ricas deixam o gráfico distorcido

dados_municipios_histograma = dados_municipios_histograma[dados_municipios_histograma['pop'] > 30000]
#vamos olhar só pra munícipios que suportam uma Renner e deixar só os passíveis de uma abertura

dados_lojas_e_economicos_histograma = dados_lojas_e_economicos[dados_lojas_e_economicos['pib_per_capta'] < 100]
#vamos tirar as cidades ricas da nossa amostra de lojas também

Agora podemos visualizar nosso gráfico.

In [ ]:
fig, ax = plt.subplots(figsize = (10, 7))

ax.hist(dados_municipios_histograma['pib_per_capta'], bins = 50)
ax.hist(dados_lojas_e_economicos_histograma['pib_per_capta'], bins = 50)
plt.xlabel('PIB Per Capita', fontsize = 15)
plt.ylabel('Número de lojas ou municípios', fontsize = 15)
ax.tick_params(axis='both', labelsize=15)
plt.title("Distribuição de Lojas Renner X \n Distribuição de Municípios no Brasil" , fontsize = 15)
ax.grid(False)
plt.show()

## Passo 4: Concluir padrões observados.

# Dois padrões:

* A maioria das lojas da Renner são em Shoppings.
* Cidades com menores PIB per capita não são alvos da empresa.

# Análise de dados e projeções

--------------

## Passo 5: Traçar um plano de análise.

* Nós sempre queremos ser conservadores em projeções.
* Vamos manter o padrão de shoppings que se encaixam na descrição socio-economica que achamos.

## Passo 6: Pegar os dados faltantes para a projeção.

https://abrasce.com.br/guia-de-shoppings/

In [ ]:
dados_shoppings = pd.read_csv('shoppings.csv')
dados_shoppings['City'] = dados_shoppings['City'].str.upper()
dados_shoppings['City'] = dados_shoppings['City'].str.strip()

dados_shoppings

## Passo 7: Fazer a projeção.

* Pegar os dados de quantos shoppings existem em uma determinada cidade.
* Pegar quantas lojas a Renner tem na mesma cidade.
* Caso o número seja menor, contar essa abertura.
* Lojas de rua vão subestimar essa projeção.
* Vamos lidar com as lojas de rua depois.

<br>

Obs: Você pode tentar cruzar os endereços em uma análise ainda mais criteriosa.

In [ ]:
shoppings_por_cidades = dados_shoppings.groupby('City')['City'].size().sort_values()

shoppings_por_cidades

In [ ]:
dados_lojas_em_shoppings = dados_lojas_e_economicos[dados_lojas_e_economicos['Shopping Mall / Street'] == 'Shopping Mall']

dados_lojas_em_shoppings['City'] = dados_lojas_em_shoppings['City'].str.upper()

dados_lojas_em_shoppings

In [ ]:
lojas_em_shoppings_por_cidade = dados_lojas_em_shoppings.groupby('City')['City'].size().sort_values()

lojas_em_shoppings_por_cidade

In [ ]:
shoppings_por_cidades = shoppings_por_cidades.to_frame()
lojas_em_shoppings_por_cidade = lojas_em_shoppings_por_cidade.to_frame()

shoppings_por_cidades.columns = ['Nº de shoppings']
lojas_em_shoppings_por_cidade.columns = ['Nº de lojas']

In [ ]:
lojas_em_shoppings_por_cidade

In [ ]:
df_lojas_x_shoppings = lojas_em_shoppings_por_cidade.join(shoppings_por_cidades, how = 'outer')

df_lojas_x_shoppings

In [ ]:
df_lojas_x_shoppings = df_lojas_x_shoppings.fillna(0)

In [ ]:
df_lojas_x_shoppings['Novas Aberturas'] = df_lojas_x_shoppings['Nº de shoppings'] - df_lojas_x_shoppings['Nº de lojas']

df_lojas_x_shoppings

In [ ]:
df_lojas_x_shoppings.sort_values("Novas Aberturas")

* Tem cidade que vai ficar negativa essa conta e precisamos tirar isso da amostra.

In [ ]:
df_lojas_x_shoppings = df_lojas_x_shoppings[df_lojas_x_shoppings['Novas Aberturas'] != -1]

df_lojas_x_shoppings

In [ ]:
df_lojas_x_shoppings['Novas Aberturas'].sum()

# Mas e as lojas de rua?

* A grande restrição são as lojas em shopping. Vamos pegar as lojas totais finais e manter a proporção em relação ao que existe hoje.

In [ ]:
lojas_totais = dados_lojas_e_economicos.groupby('Shopping Mall / Street')['Shopping Mall / Street'].size()

lojas_totais

In [ ]:
lojas_totais['Shopping Mall'] = lojas_totais['Shopping Mall']  + df_lojas_x_shoppings['Novas Aberturas'].sum()

In [ ]:
lojas_totais

In [ ]:
lojas_rua_lojas_shop = 638/0.875878
lojas_rua_lojas_shop

In [ ]:
aberturas_rua = lojas_rua_lojas_shop - lojas_totais['Shopping Mall'] - lojas_totais['Street']
aberturas_rua

# Aberturas da Renner

* 264 Lojas em shoppings
* 37 lojas na rua
* 301 lojas no total!
* Não estamos considerando novos shoppings.
* Um aspecto importante é ver um teto claro pra Renner nesse perfil de shoppings que existe hoje.
* A renner abre aprox 25 lojas por ano.

## Acabou? Claro que não!